In [1]:
import numpy as np
import sys
sys.path.append("../code/")
from structure import *
from algo import *
from basic import *
from analysis import *
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from autograd.numpy import linalg as LA
import pickle
import itertools
from itertools import product
import calendar
from sklearn.metrics import mean_squared_error
%load_ext autoreload
%autoreload 2
import warnings
import seaborn as sns
warnings.filterwarnings("ignore")
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
import numpy as np

In [2]:
year = 2015
dataset = 'artificial'
order = APP_NAME_OBSERVED

In [3]:
# load the selected pairs with the best parameter
sp = {}
param = np.load("../data/best-param-{}-active.npy".format(year)).item()
k, latent_dimension, lambda1, lambda2, lambda3, kernel, delta = param.split("-")
filename = "sp-pre-none-prev_future_weighted-{}-{}-0.1-0.1-0.0-{}-{}-{}-{}-{}".format(kernel, delta, lambda1, lambda2,
                                                                                    lambda3, k, latent_dimension)
directory = "../data/result/reg/update_season/active/{}/{}/".format(year, dataset)
full_path = directory + filename
sp['active'] = np.load(full_path)


param = np.load("../data/best-param-{}-random.npy".format(year)).item()
k, latent_dimension, lambda1, lambda2, lambda3 = param.split("-")
filename = "sp-pre-{}-{}-{}-{}-{}".format(lambda1, lambda2, lambda3, k, latent_dimension)
directory = "../data/result/reg/update_season/random/{}/{}/".format(year, dataset)
full_path = directory + filename
sp['random'] = np.load(full_path)


param = np.load("../data/best-param-{}-qbc.npy".format(year)).item()
k, latent_dimension, lambda1, lambda2, lambda3 = param.split("-")
filename = "sp-pre-{}-{}-{}-{}-{}".format(lambda1, lambda2, lambda3, k, latent_dimension)
directory = "../data/result/reg/update_season/qbc/{}/{}/".format(year, dataset)
full_path = directory + filename
sp['qbc'] = np.load(full_path)


param = np.load("../data/best-param-{}-vb-new.npy".format(year)).item()
k, latent_dimension, a_gamma, b_gamma, a_beta, b_beta, a_alpha, b_alpha, random_seed = param.split("-")
filename = "sp-{}-{}-{}-{}-{}-{}-{}-{}".format(k, latent_dimension, a_gamma, b_gamma, a_beta, b_beta, a_alpha, b_alpha)
directory = "../data/result/vb_var/{}/{}/".format(year, dataset)
full_path = directory + filename
sp['var'] = np.load(full_path)

In [21]:
random_seed = 0
fold_num = 0
month = 0
select_id = 0
home_index = sp['active'][random_seed][fold_num][month][select_id][0]
tensor, hids = get_tensor(2015, 'artificial')
train, test, tr_ids, tt_ids = get_train_test(tensor, hids, fold_num=fold_num)

In [28]:
raw_data = np.load("../data/data-2013-2017-observed-filtered-artificial.npy").item()

In [105]:
survey = pd.read_csv("../data/survey_2013_all_participants_modified.csv")
metadata = pd.read_csv("../data/metadata.csv")

In [101]:
square = {}


In [111]:
for method in sp.keys():
    square[method] = {}
    for t in range(12):
        s = 0
        c = 0
        for random_seed in range(5):
            for fold_num in range(5):
                train, test, tr_ids, tt_ids = get_train_test(tensor, hids, fold_num=fold_num)

                for i in range(5):
                    home_index = sp[method][random_seed][fold_num][t][i][0]
                    train, test, tr_ids, tt_ids = get_train_test(tensor, hids, fold_num=fold_num)
                    hid = tr_ids[home_index]
    #                 print(hid)
                    info = metadata.loc[metadata['dataid'] == hid]
    #                 info = survey.loc[survey['dataid' == hid]]
    #                 print(info)

                    try:
    #                     print(t, int(info['num_people']))
                        s += float(info['house_construction_year'])
    #                     s += int(info['num_people'])
                        c += 1
                    except:
                        continue
        if c == 0:
            print(t, "Null")
        else:
            square[method][t] = s/c
            print(t, s/c)

0 nan
1 2008.864
2 2006.696
3 2008.032
4 2008.464
5 2001.28
6 2006.536
7 2004.44
8 2004.136
9 2006.912
10 nan
11 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
0 nan
1 2007.728
2 2006.856
3 nan
4 nan
5 2002.016
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
0 nan
1 2008.888
2 2005.848
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan


In [110]:
pd.DataFrame(square).mean()

active    2425.384333
qbc       2295.690333
random    2138.829000
var       2155.671333
dtype: float64

In [64]:
info = survey.loc[survey['dataid'] == hid]
# info['num_people']
info

,dataid,number_floors,house_num_rooms,house_square_feet,sex_males,sex_females,num_people
97,3723,2,3,1551,1,1.0,2
